In [51]:
import os
# Change working directory
os.chdir('/Users/suongsuong/Documents/GitHub/Reactivity-based-metric-of-complexity/Reduction of ketone/')

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split,KFold, cross_validate, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

# Import data

In [6]:
df_features = pd.read_excel('ReductionKetone_withFeatures.xlsx')

In [11]:
df_features = pd.read_excel('ReductionKetone_withFeatures.xlsx')

yield_features_columns = ['Yield (number)', 
       # Basic features of molecule
       'Reactant MW', 'Ring', 'aroma_Ring', 'Carbons',
       'Chiral Carbons', 'sp3 Carbons', 'HA', 'Number of ketone',
       # Features of C carbonyl
       'Partial Charge of C carbonyl', 'L',
       'Bmin', 'Bmax', 'PBV',
       # Features of alpha positions
       'Number aromatic ring at alpha', 'Number of sub group both side',  'L_alpha Sum',
       'Bmin_alpha Sum', 'Bmax_alpha Sum', 
       'PBV_alpha Sum', 'PBV_alpha Diff.',
       # Published indexes
       'Cm', 'Cm/HA', 'SPS', 'nSPS', 'F_sp3',
       'F_Cstereo', 'C_T', 'Harary', 'GraphDistance', 'Diameter', 'Platt',
       'SimpleTopo', 'GeometricTopo', 'ArithmeticTopo', 'MolSizeTotalInf']

features_col_yield = df_features[yield_features_columns]

# Single correlation

In [17]:
corr = features_col_yield.corr()

yield_corr = corr['Yield (number)'].drop('Yield (number)') 
yield_corr_df = pd.DataFrame({
    'Feature': yield_corr.index,
    'Correlation with Yield': np.round(yield_corr.values,3)
})
yield_corr_df = yield_corr_df.sort_values(by='Correlation with Yield', ascending=False)
yield_corr_df


,Feature,Correlation with Yield
7,Number of ketone,0.193
16,Bmin_alpha Sum,0.111
25,F_Cstereo,0.069
10,Bmin,0.069
24,F_sp3,0.037
19,PBV_alpha Diff.,0.027
4,Chiral Carbons,0.021
6,HA,0.009
8,Partial Charge of C carbonyl,-0.009
20,Cm,-0.032


# Split test and training set

In [40]:
# Normalize to 0 and 1
scaler = MinMaxScaler()
model=scaler.fit(features_col_yield)
scaled_features_col_yield=model.transform(features_col_yield)

# Convert the scaled array back to a DataFrame
scaled_features_col_yield_df = pd.DataFrame(
    scaled_features_col_yield, 
    columns=features_col_yield.columns
)

In [42]:
X = scaled_features_col_yield_df.drop(columns = 'Yield (number)')
y = scaled_features_col_yield_df['Yield (number)']

# Split the data into train:test = 0.8:0.2
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

print("x train dimensions :",X_train.shape)
print("x test dimensions: ",X_test.shape)
print("y train dimensions :",y_train.shape)
print("y test dimensions :",y_test.shape)

x train dimensions : (726, 35)
x test dimensions:  (182, 35)
y train dimensions : (726,)
y test dimensions : (182,)


# Model

In [44]:
def cal_within_range_error(threshold, y_test, y_pred):
    within_range = np.abs(y_test - y_pred) <= threshold
    proportion_within_range = np.mean(within_range) * 100
    return proportion_within_range

def evaluate(random_search, X_train, X_test, y_train, y_test):
    # Get the best parameters and model
    best_params = random_search.best_params_
    best_model = random_search.best_estimator_

    # Make predictions
    y_train_pred = best_model.predict(X_train)
    y_test_pred = best_model.predict(X_test)

    # Calculate metrics
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    train_within_range_error = cal_within_range_error(0.05, y_train, y_train_pred)
    test_within_range_error = cal_within_range_error(0.05, y_test, y_test_pred)

    # Print results
    print(f"Best Parameters: {best_params}")
    print(f"Train R2 Value: {train_r2}")
    print(f"Test R2 Value: {test_r2}")
    print(f"Train MAE Value: {train_mae}")
    print(f"Test MAE Value: {test_mae}")
    print(f"Train RMSE Value: {train_rmse}")
    print(f"Test RMSE Value: {test_rmse}")
    print(f"Train % within 5% error: {train_within_range_error}")
    print(f"Test % within 5% error: {test_within_range_error}")

In [45]:
models = {
        'Linear Regression': LinearRegression(),
        'Decision Tree Regressor': DecisionTreeRegressor(),
        'Random Forest Regressor': RandomForestRegressor(),
        'Ridge Regressor': Ridge(alpha=1.0),
        'Lasso Regressor': Lasso(alpha=0.1, max_iter=10000), 
        'SVR': SVR(kernel='rbf', C=1.0, epsilon=0.2),
        'Gradient Boosting Regressor': GradientBoostingRegressor(n_estimators=100, learning_rate=0.1),
    }

In [48]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)

Avg_TrainR2Value = []
Avg_TestR2Value = []
Avg_TrainMAEValue = []
Avg_TestMAEValue = []
Avg_TrainRMSEValue = []
Avg_TestRMSEValue = []
Avg_WithinRangeError = []
Model_name = []
threshold = 0.05

for name, model in models.items():
    Model_name.append(name)
    
    # Cross-validation
    CrossValidate = cross_validate(model, X_train, y_train, cv=cv, return_train_score=True, scoring=None)

    # Fit model and get predictions
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    within_range_error = cal_within_range_error(threshold, y_test, y_test_pred)
    
    Avg_TrainR2Value.append(train_r2)
    Avg_TestR2Value.append(test_r2)
    Avg_TrainMAEValue.append(train_mae)
    Avg_TestMAEValue.append(test_mae)
    Avg_TrainRMSEValue.append(train_rmse)
    Avg_TestRMSEValue.append(test_rmse)
    Avg_WithinRangeError.append(within_range_error)

# Create a summary DataFrame
summary_models_df = pd.DataFrame({
    'Model': Model_name,
    'Train R2 Value': Avg_TrainR2Value,
    'Test R2 Value': Avg_TestR2Value,
    'Train MAE Value': Avg_TrainMAEValue,
    'Test MAE Value': Avg_TestMAEValue,
    'Train RMSE Value': Avg_TrainRMSEValue,
    'Test RMSE Value': Avg_TestRMSEValue,
    'Within 5% Error': Avg_WithinRangeError
})

summary_models_df

,Model,Train R2 Value,Test R2 Value,Train MAE Value,Test MAE Value,Train RMSE Value,Test RMSE Value,Within 5% Error
0,Linear Regression,0.262212,0.191782,1.239983e-01,0.148155,1.666863e-01,0.190745,20.879121
1,Decision Tree Regressor,1.000000,-0.122293,9.175397e-19,0.164033,1.009294e-17,0.224771,23.626374
2,Random Forest Regressor,0.900146,0.403720,4.505855e-02,0.123842,6.132221e-02,0.163837,25.824176
3,Ridge Regressor,0.222980,0.209301,1.276705e-01,0.147237,1.710607e-01,0.188666,20.879121
4,Lasso Regressor,0.000000,-0.024231,1.483439e-01,0.168273,1.940592e-01,0.214727,14.285714
5,SVR,0.370306,0.338290,1.278691e-01,0.139800,1.539923e-01,0.172592,23.626374
6,Gradient Boosting Regressor,0.747878,0.351223,7.420390e-02,0.132363,9.744051e-02,0.170898,20.879121


# tunning hyperparameters

#### Random Forest Regressor

In [61]:
pca = PCA()
rf = RandomForestRegressor()

# Create the pipeline
model = Pipeline(steps=[('pca', pca), ('rf', rf)])

# Define the parameter grid
param_grid = {
    'pca__n_components': [None, 10, 20, 30],  # Parameters for PCA
    'rf__n_estimators': [100, 200, 300, 400, 500],  # Parameters for RandomForestRegressor
    'rf__max_features': ['sqrt', 'log2', None, 0.5, 0.75],
    'rf__max_depth': [None, 10, 20, 30, 40, 50],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4],
    'rf__bootstrap': [True, False]
}

# Set up the randomized search
random_search_rf = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                                    n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1)

# Fit the model
random_search_rf.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END pca__n_components=20, rf__bootstrap=False, rf__max_depth=20, rf__max_features=log2, rf__min_samples_leaf=2, rf__min_samples_split=10, rf__n_estimators=100; total time=   0.5s
[CV] END pca__n_components=20, rf__bootstrap=False, rf__max_depth=20, rf__max_features=log2, rf__min_samples_leaf=2, rf__min_samples_split=10, rf__n_estimators=100; total time=   0.5s
[CV] END pca__n_components=20, rf__bootstrap=False, rf__max_depth=20, rf__max_features=log2, rf__min_samples_leaf=2, rf__min_samples_split=10, rf__n_estimators=100; total time=   0.5s
[CV] END pca__n_components=20, rf__bootstrap=False, rf__max_depth=20, rf__max_features=log2, rf__min_samples_leaf=2, rf__min_samples_split=10, rf__n_estimators=100; total time=   0.5s
[CV] END pca__n_components=20, rf__bootstrap=False, rf__max_depth=20, rf__max_features=log2, rf__min_samples_leaf=2, rf__min_samples_split=10, rf__n_estimators=100; total time=   0.5s
[CV] END pca__n_c

In [ ]:
evaluate(random_search_rf, X_train, X_test, y_train, y_test)

Best Parameters: {'rf__n_estimators': 300, 'rf__min_samples_split': 2, 'rf__min_samples_leaf': 1, 'rf__max_features': 'sqrt', 'rf__max_depth': 30, 'rf__bootstrap': False, 'pca__n_components': None}
Train R2 Value: 0.99999994678488
Test R2 Value: 0.3050859179472124
Train MAE Value: 1.3352419064521291e-05
Test MAE Value: 0.1341274802301596
Train RMSE Value: 4.476635085528891e-05
Test RMSE Value: 0.17686968341674467
Train RMSE Value: 100.0
Test RMSE Value: 24.725274725274726


#### Gradient Boosting Regressor

In [ ]:
gb = GradientBoostingRegressor()

# Define the parameter grid
param_grid = {
    'pca__n_components': [None, 10, 20, 30],  # Parameters for PCA
    'n_estimators': [100, 200, 300, 400, 500],
    'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
    'max_depth': [3, 4, 5, 6, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 0.9, 1.0]
}

# Set up the randomized search
random_search_gb = RandomizedSearchCV(estimator=gb, param_distributions=param_grid,
                                    n_iter=100, cv=5, verbose=2, random_state=42, n_jobs=-1)

# Fit the model
random_search_gb.fit(X_train, y_train)

In [ ]:
evaluate(random_search_gb, X_train, X_test, y_train, y_test)